# 모델 저장하기

- 무엇을 저장해야 할까?
    - 모델의 구조
    - 모델의 weights

## 훈련하는 동안 체크포인트 저장하기

In [7]:
import os

import tensorflow as tf
from tensorflow import keras

In [8]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [9]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [10]:
model = create_model()

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 체크포인트 콜백 사용하기

In [11]:
# 체크포인트 파일 경로 설정
checkpoint_path = "training_1/cp.ckpt" 
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'training_1'

In [12]:
# 모델의 가중치를 저장하는 체크포인트 콜백 선언
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백 전달

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.3160 - accuracy: 0.6022
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 3s 24ms/step - loss: 1.2029 - accuracy: 0.6420 - val_loss: 0.7173 - val_accuracy: 0.7850
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4257 - accuracy: 0.8813
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.4227 - accuracy: 0.8820 - val_loss: 0.5508 - val_accuracy: 0.8290
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2736 - accuracy: 0.9255
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 0.2915 - accuracy: 0.9210 - val_loss: 0.4799 - val_accuracy: 0.8470
Epoch 4/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2111 - accuracy: 0.9531
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - lo

In [13]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다  
 - 아키텍처가 동일한 모델 간에는 가중치를 공유 가능 
 
 - 그렇다면, 원래 모델과 동일한 아키텍처를 가진 모델을 만든 다음 체크포인트 부여해보자.

In [14]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3406 - accuracy: 0.1150 - 286ms/epoch - 9ms/step
훈련되지 않은 모델의 정확도: 11.50%


허접하다.

In [15]:
# 체크포인트에서 가중치를 로드하고 평가해보자.
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4013 - accuracy: 0.8740 - 88ms/epoch - 3ms/step
복원된 모델의 정확도: 87.40%


In [16]:
# 파일 이름에 epoch 값 부여 (format 형식)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

- 체크포인트 주요 파라미터 

In [17]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 주기 설정 가능 ('epoch'=에폭마다, 숫자=해당 배치마다)
    )

In [18]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [19]:
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 19ms/step - loss: 1.2035 - accuracy: 0.6680 - val_loss: 0.6999 - val_accuracy: 0.8030
Epoch 2/50
32/32 [==============================] - 0s 14ms/step - loss: 0.4247 - accuracy: 0.8860 - val_loss: 0.5203 - val_accuracy: 0.8460
Epoch 3/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2937 - accuracy: 0.9150 - val_loss: 0.4563 - val_accuracy: 0.8640
Epoch 4/50
32/32 [==============================] - 0s 12ms/step - loss: 0.2032 - accuracy: 0.9510 - val_loss: 0.4587 - val_accuracy: 0.8570
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 0.1610 - accuracy: 0.9630 - val_loss: 0.4748 - val_accuracy: 0.8480
Epoch 6/50
32/32 [==============================] - 0s 13ms/step - loss: 0.1217 - accuracy: 0.9770 - val_loss: 0.4306 - val_accuracy: 0.8630
Epoch 7/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0947 - accuracy: 0.9840 - val_loss: 0.4023 - val_accuracy: 0.8610
Epoch 8/50
32

In [20]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3590 - accuracy: 0.1170 - 410ms/epoch - 13ms/step
훈련되지 않은 모델의 정확도: 11.70%


In [21]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0007.ckpt'

In [22]:
model.load_weights(latest)

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4023 - accuracy: 0.8610 - 133ms/epoch - 4ms/step
훈련되지 않은 모델의 정확도: 86.10%


In [23]:
### mytraining1 폴더에 저장
### best only로 저장, 최종 값만 저장되도록
### weight값만
checkpoint_path = "mytraining1/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 16ms/step - loss: 1.1642 - accuracy: 0.6850 - val_loss: 0.7161 - val_accuracy: 0.7810
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 0.4357 - accuracy: 0.8740 - val_loss: 0.5374 - val_accuracy: 0.8320
Epoch 3/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2795 - accuracy: 0.9400 - val_loss: 0.4872 - val_accuracy: 0.8410
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2217 - accuracy: 0.9410 - val_loss: 0.4752 - val_accuracy: 0.8430
Epoch 5/50
32/32 [==============================] - 0s 13ms/step - loss: 0.1612 - accuracy: 0.9640 - val_loss: 0.4380 - val_accuracy: 0.8600
Epoch 6/50
32/32 [==============================] - 0s 13ms/step - loss: 0.1095 - accuracy: 0.9830 - val_loss: 0.4276 - val_accuracy: 0.8650
Epoch 7/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0830 - accuracy: 0.9890 - val_loss: 0.4582 - val_accuracy: 0.8480
Epoch 8/50
32

In [24]:
model.load_weights(checkpoint_path)

model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.3965 - accuracy: 0.8670 - 88ms/epoch - 3ms/step


[0.39647623896598816, 0.8669999837875366]

In [25]:
tf.keras.callbacks.ModelCheckpoint?

In [26]:
### mytraining2 폴더에 저장
### 5epoch마다 저장
### weight값만
checkpoint_path = "mytraining3/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=False, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    period=5, # 체크포인트 저장하는 에폭주기 설정 가능
    save_freq='epoch' # epoch의 끝에서 저장할지 여부
    )

In [27]:
model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 16ms/step - loss: 1.1753 - accuracy: 0.6650 - val_loss: 0.8007 - val_accuracy: 0.7440
Epoch 2/50
32/32 [==============================] - 0s 15ms/step - loss: 0.4266 - accuracy: 0.8750 - val_loss: 0.5341 - val_accuracy: 0.8220
Epoch 3/50
32/32 [==============================] - 0s 15ms/step - loss: 0.2934 - accuracy: 0.9220 - val_loss: 0.4678 - val_accuracy: 0.8480
Epoch 4/50
32/32 [==============================] - 0s 16ms/step - loss: 0.2073 - accuracy: 0.9570 - val_loss: 0.4377 - val_accuracy: 0.8540
Epoch 5/50
32/32 [==============================] - 1s 17ms/step - loss: 0.1508 - accuracy: 0.9660 - val_loss: 0.4414 - val_accuracy: 0.8550
Epoch 6/50
32/32 [==============================] - 1s 16ms/step - loss: 0.1153 - accuracy: 0.9800 - val_loss: 0.4279 - val_accuracy: 0.8560
Epoch 7/50
32/32 [==============================] - 1s 21ms/step - loss: 0.0978 - accuracy: 0.9860 - val_loss: 0.4478 - val_accuracy: 0.8510
Epoch 8/50
32

### early stopping & 체크포인트 같이 사용하기

In [28]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [29]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [30]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [31]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [32]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback, early], # 체크포인트 부여, early stopping
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 23ms/step - loss: 1.1496 - accuracy: 0.6670 - val_loss: 0.7448 - val_accuracy: 0.7620
Epoch 2/100
32/32 [==============================] - 0s 15ms/step - loss: 0.4130 - accuracy: 0.8840 - val_loss: 0.5193 - val_accuracy: 0.8400
Epoch 3/100
32/32 [==============================] - 1s 17ms/step - loss: 0.2714 - accuracy: 0.9260 - val_loss: 0.4820 - val_accuracy: 0.8560
Epoch 4/100
32/32 [==============================] - 0s 15ms/step - loss: 0.2043 - accuracy: 0.9470 - val_loss: 0.4419 - val_accuracy: 0.8660
Epoch 5/100
32/32 [==============================] - 1s 16ms/step - loss: 0.1541 - accuracy: 0.9680 - val_loss: 0.4256 - val_accuracy: 0.8720
Epoch 6/100
32/32 [==============================] - 0s 14ms/step - loss: 0.1267 - accuracy: 0.9740 - val_loss: 0.4250 - val_accuracy: 0.8670
Epoch 7/100
32/32 [==============================] - 0s 14ms/step - loss: 0.0788 - accuracy: 0.9890 - val_loss: 0.4042 - val_accuracy: 0.8710
Epoch 

### 직접 가중치 체크포인트 저장

In [33]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint.ckpt')

model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint.ckpt')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4047 - accuracy: 0.8730 - 278ms/epoch - 9ms/step
복원된 모델의 정확도: 87.30%


## 전체 모델 저장(모델레이어와 함께 저장)
1) save_weights_only=False 로 변경  
2) 직접 model.save() : SavedModel(pb)  
3) HDF5 파일로 저장하기

### save_weights_only=False 로 변경

In [34]:
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [35]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [36]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [37]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 1.1515 - accuracy: 0.6720

32/32 [==============================] - 2s 53ms/step - loss: 1.1515 - accuracy: 0.6720 - val_loss: 0.7102 - val_accuracy: 0.7950
Epoch 2/100
31/32 [============================>.] - ETA: 0s - loss: 0.4200 - accuracy: 0.8831

32/32 [==============================] - 1s 33ms/step - loss: 0.4171 - accuracy: 0.8840 - val_loss: 0.5206 - val_accuracy: 0.8390
Epoch 3/100
32/32 [==============================] - ETA: 0s - loss: 0.2838 - accuracy: 0.9320

32/32 [==============================] - 1s 33ms/step - loss: 0.2838 - accuracy: 0.9320 - val_loss: 0.4994 - val_accuracy: 0.8360
Epoch 4/100
30/32 [===========================>..] - ETA: 0s - loss: 0.2066 - accuracy: 0.9531

32/32 [==============================] - 1s 33ms/step - loss: 0.2135 - accuracy: 0.9490 - val_loss: 0.4222 - val_accuracy: 0.8620
Epoch 5/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1549 - accuracy: 0.9600 - val_loss: 0.4462 - val_accuracy: 0.8550
Epoch 6/100
32/32 [==============================] - ETA: 0s - loss: 0.1167 - accuracy: 0.9750

...

32/32 [==============================] - 1s 36ms/step - loss: 0.0882 - accuracy: 0.9810 - val_loss: 0.3935 - val_accuracy: 0.8760
Epoch 8/100
32/32 [==============================] - 1s 17ms/step - loss: 0.0614 - accuracy: 0.9970 - val_loss: 0.3979 - val_accuracy: 0.8720
Epoch 9/100
29/32 [==========================>...] - ETA: 0s - loss: 0.0486 - accuracy: 0.9968

32/32 [==============================] - 1s 45ms/step - loss: 0.0485 - accuracy: 0.9970 - val_loss: 0.3902 - val_accuracy: 0.8770
Epoch 10/100
32/32 [==============================] - 0s 14ms/step - loss: 0.0398 - accuracy: 0.9990 - val_loss: 0.3933 - val_accuracy: 0.8780
Epoch 11/100
32/32 [==============================] - ETA: 0s - loss: 0.0279 - accuracy: 1.0000

32/32 [==============================] - 1s 30ms/step - loss: 0.0279 - accuracy: 1.0000 - val_loss: 0.3887 - val_accuracy: 0.8830
Epoch 12/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0258 - accuracy: 1.0000 - val_loss: 0.3914 - val_accuracy: 0.8740
Epoch 13/100
32/32 [==============================] - ETA: 0s - loss: 0.0220 - accuracy: 1.0000

32/32 [==============================] - 1s 31ms/step - loss: 0.0220 - accuracy: 1.0000 - val_loss: 0.3870 - val_accuracy: 0.8810
Epoch 14/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0199 - accuracy: 1.0000 - val_loss: 0.4101 - val_accuracy: 0.8740
Epoch 15/100
30/32 [===========================>..] - ETA: 0s - loss: 0.0147 - accuracy: 1.0000

32/32 [==============================] - 1s 35ms/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 0.3867 - val_accuracy: 0.8840
Epoch 16/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 0.4045 - val_accuracy: 0.8800
Epoch 17/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.3966 - val_accuracy: 0.8830
Epoch 18/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.4097 - val_accuracy: 0.8810
Epoch 19/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.4126 - val_accuracy: 0.8780
Epoch 20/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.4078 - val_accuracy: 0.8810
Epoch 21/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.4245 - val_accuracy: 0.8810
Epoch 22/100

### SavedModel(pb)
SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함

In [38]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1449 - accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4241 - accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2804 - accuracy: 0.9350
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1972 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1480 - accuracy: 0.9660


In [39]:
# my_model 디렉토리
!ls saved_model

my_model


In [40]:
# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [41]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [42]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4382 - accuracy: 0.8550 - 200ms/epoch - 6ms/step
복원된 모델의 정확도: 85.50%
32/32 [==============================] - 1s 3ms/step
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [43]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1971 - accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4311 - accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.3124 - accuracy: 0.9140
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2183 - accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1612 - accuracy: 0.9650


### h5 VS pb 
- 공통점 : 레이어 & 가중치 모두 저장
- 차이점  
1) HDF5 : 객체 구성을 사용하여 모델 아키텍처를 저장  
2) SavedModel : 실행 그래프를 저장. 원본 코드 없이도 사용자 지정 객체를 저장 가능

#### h5 to pb 변환하기

In [44]:
!mkdir converted

mkdir: cannot create directory ‘converted’: File exists


In [45]:
model = tf.keras.models.load_model('my_model.h5')

In [46]:
export_path = 'converted'
model.save(export_path, save_format="tf")